In [1]:
import os

import geopandas
import matplotlib as mpl
import pandas as pd
import pydeck

In [2]:
MAPBOX_API_KEY = os.environ["MAPBOX_API_KEY"]

In [3]:
MAX_BUILDINGS = 6000

In [4]:
start_point = (6.1402,46.208742)

In [5]:
delta = 0.001
bbox =  (start_point[0] - delta, start_point[1]-delta, start_point[0] + delta, start_point[1]+delta) 

In [6]:
from ipybuilding.datasets import geneva

base_path, facade_path, roof_path = geneva.load_data()

In [7]:
bases = geopandas.read_file(base_path)

facade = geopandas.read_file(facade_path)

toits = geopandas.read_file(roof_path)


bases.columns = [s.lower() for s in bases.columns]
facade.columns = [s.lower() for s in facade.columns]
toits.columns = [s.lower() for s in toits.columns]

In [8]:
path = 'Geneva/SCANE_INDICE_DERNIER.csv'
heat_demand = pd.read_csv(path, sep=';')
heat_demand.columns = [s.lower() for s in heat_demand.columns]
heat_demand.egid = heat_demand.egid.astype(int)

In [9]:
bases = bases.merge(heat_demand, on='egid')
facade = facade.merge(heat_demand, on='egid')
toits = toits.merge(heat_demand, on='egid')

In [10]:
colourmap_name = 'RdYlGn_r'

In [11]:
norm = mpl.colors.Normalize(vmin=1, vmax=bases.indice.quantile(0.95))
cmap = mpl.cm.get_cmap(colourmap_name)

cmap_val = cmap(norm(bases.indice)) 

bases['color'] = (cmap_val* 255).astype(int).tolist()

In [12]:
norm = mpl.colors.Normalize(vmin=1, vmax=toits.indice.quantile(0.95))
cmap = mpl.cm.get_cmap(colourmap_name)

cmap_val = cmap(norm(toits.indice)) 

toits['color'] = (cmap_val* 255).astype(int).tolist()


In [13]:
norm = mpl.colors.Normalize(vmin=1, vmax=facade.indice.quantile(0.95))
cmap = mpl.cm.get_cmap(colourmap_name)

cmap_val = cmap(norm(facade.indice)) 

facade['color'] = (cmap_val* 255).astype(int).tolist()

In [14]:
keep_columns = ['egid', 'geometry', 'indice', 'color']
bases = bases[keep_columns]
facade = facade[keep_columns]
toits = toits[keep_columns]

In [15]:
def filtered_buildings_geom(bbox):
    return (bases.cx[bbox[0]: bbox[2], bbox[1]:bbox[3]],
            facade.cx[bbox[0]: bbox[2], bbox[1]:bbox[3]],
            toits.cx[bbox[0]: bbox[2], bbox[1]:bbox[3]])

In [16]:
f_base, f_face, f_toit = filtered_buildings_geom(bbox)


In [17]:
coords = f_base.centroid.map(lambda p: [p.x, p.y])

INITIAL_VIEW_STATE = pydeck.data_utils.compute_view(coords.values)
INITIAL_VIEW_STATE.zoom = 16
INITIAL_VIEW_STATE.max_zoom = 16


/var/folders/23/f82vmc8x4d18zwnnbl6dxvnw0000gn/T/ipykernel_93360/3829069764.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = f_base.centroid.map(lambda p: [p.x, p.y])


In [18]:
# AWS Open Data Terrain Tiles
TERRAIN_IMAGE = "https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png"

# Define how to parse elevation tiles
ELEVATION_DECODER = {"rScaler": 256, "gScaler": 1, "bScaler": 1 / 256, "offset": -32768}

SURFACE_IMAGE = f"https://api.mapbox.com/v4/mapbox.satellite/{{z}}/{{x}}/{{y}}@2x.png?access_token={MAPBOX_API_KEY}"


terrain_layer = pydeck.Layer(
    "TerrainLayer", elevation_decoder=ELEVATION_DECODER, texture=SURFACE_IMAGE, elevation_data=TERRAIN_IMAGE,
    # light_settings=lights

)


bases_layer = pydeck.Layer('GeoJsonLayer', 
                           data=f_base, get_fill_color=[255,100,255], pickable=True
                          )
facade_layer = pydeck.Layer('GeoJsonLayer', 
                            data=f_face, 
                            get_fill_color='color', 
                            line_width_scale=0.1,
                            pickable=True,
                            filled=True,
                            wireframe=True,
                           )

toits_layer = pydeck.Layer('GeoJsonLayer', 
                           data=f_toit, 
                           get_fill_color='color', 
                           pickable=True,
                           stroked=False,
                           filled=True,
                           wireframe=True,
                      )

r = pydeck.Deck(
    layers=[
        terrain_layer,
        bases_layer, 
        facade_layer, 
        toits_layer
    ],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style='satellite',
    tooltip={
        'text': 'IDC: {indice} MJ/m2·a'
    }
)


def filter_by_viewport(widget_instance, payload):
    try:
        west_lng, north_lat = payload['data']['nw']
        east_lng, south_lat = payload['data']['se']
        pad = 0.002
        bbox = [east_lng - pad, south_lat -pad, west_lng+pad, north_lat+pad, ]
        
        f_base, f_face, f_toit = filtered_buildings_geom(bbox)
        txt = f'Buildings in view ({east_lng:.2f} {south_lat:.2f} {west_lng:.2f} {north_lat:.2f}): {f_base.egid.nunique()}'
        if f_base.egid.nunique() > MAX_BUILDINGS:
            # dont try to show too much stuff!
            txt += ' TOO MANY BUILDINGS TO SHOW, TRY ZOOMING IN'
            pass
        else:
        
            bases_layer.data = f_base
            facade_layer.data = f_face
            toits_layer.data = f_toit
        
        text.value = txt
        
        r.update()
    except Exception as e:
        text.value = 'Error: %s' % e

from ipywidgets import HTML

text = HTML(value=f'Buildings in viewport: { f_base.egid.nunique()}')

r.deck_widget.on_view_state_change(filter_by_viewport)
display(text)

r.show()

HTML(value='Buildings in viewport: 20')

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…